#### COLOR IMAGE

#### MARKER-BASED

In [16]:
import cv2
import numpy as np
import math
import os

# Input and Output Directories
dataset_dir = r"C:\PJT2\Orbbec_Dataset_New\Orbbec_Dataset\validation\marker-based"
output_dir = r"C:\PJT2\Orbbec_Dataset_New\Orbbec_Dataset\validation\marker-based-output"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define red color range in HSV
lower_red1 = np.array([0, 140, 70])   
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 140, 70]) 
upper_red2 = np.array([180, 255, 255])

# Morphological kernel
kernel = np.ones((5, 5), np.uint8)

# Function to calculate angle dynamically
def calculate_angle(a, b, c):
    vector_a = np.array([a[0] - b[0], a[1] - b[1]])
    vector_b = np.array([c[0] - b[0], c[1] - b[1]])
    dot_product = np.dot(vector_a, vector_b)
    magnitude_a = np.linalg.norm(vector_a)
    magnitude_b = np.linalg.norm(vector_b)
    angle_rad = np.arccos(dot_product / (magnitude_a * magnitude_b))
    return np.degrees(angle_rad)

# Function to identify correct hip, knee, and ankle
def identify_body_points(marker_coordinates):
    if len(marker_coordinates) < 3:
        return None, None, None
    
    sorted_markers = sorted(marker_coordinates, key=lambda p: p[1])
    
    hip, knee, ankle = sorted_markers[:3]
    if np.linalg.norm(np.array(knee) - np.array(ankle)) < np.linalg.norm(np.array(hip) - np.array(knee)):
        hip, knee, ankle = hip, ankle, knee  
    
    return hip, knee, ankle

# Process each image
def process_images():
    for filename in os.listdir(dataset_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(dataset_dir, filename)
            image = cv2.imread(image_path)
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            
            # Create mask for red color
            mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
            mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
            mask = mask1 + mask2
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            
            # Find contours
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            marker_coordinates = []
            
            for contour in contours:
                area = cv2.contourArea(contour)
                if area < 30 or area > 6000:
                    continue
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h
                if aspect_ratio < 0.2 or aspect_ratio > 4.0:
                    continue
                if y < 50:
                    continue
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    marker_coordinates.append((cx, cy))
            
            print(f"{filename} - Valid Marker Coordinates: {marker_coordinates}")
            hip, knee, ankle = identify_body_points(marker_coordinates)
            
            if hip and knee and ankle:
                knee_angle = calculate_angle(hip, knee, ankle)
                print(f"{filename} - Knee Angle: {knee_angle:.2f}°")

                # Draw black thin lines
                cv2.line(image, hip, knee, (0, 0, 0), 2)
                cv2.line(image, knee, ankle, (0, 0, 0), 2)

                # Draw an arc to represent the knee angle
                radius = 25
                start_angle = int(math.degrees(math.atan2(hip[1] - knee[1], hip[0] - knee[0])))
                end_angle = int(math.degrees(math.atan2(ankle[1] - knee[1], ankle[0] - knee[0])))
                cv2.ellipse(image, knee, (radius, radius), 0, start_angle, end_angle, (0, 0, 0), 2)

                # Draw rectangles around detected markers (black and thin)
                for cx, cy in marker_coordinates:
                    cv2.rectangle(image, (cx - 5, cy - 5), (cx + 5, cy + 5), (0, 0, 0), 1)

                # Display coordinates with a black background and white text
                for cx, cy in marker_coordinates:
                    text = f"({cx}, {cy})"
                    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)[0]
                    text_x, text_y = cx + 10, cy - 10
                    cv2.rectangle(image, (text_x - 2, text_y - text_size[1] - 2), (text_x + text_size[0] + 2, text_y + 2), (0, 0, 0), -1)
                    cv2.putText(image, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

                # Display knee angle with deep red text (no background)
                angle_text = f"{knee_angle:.2f} deg"
                cv2.putText(image, angle_text, (knee[0] + 25, knee[1] + 20), 
                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (20, 20, 180), 2) 

            # Save the processed image
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, image)

process_images()


image1_marker.png - Valid Marker Coordinates: [(390, 411), (565, 382), (287, 277)]
image1_marker.png - Knee Angle: 118.14°
image2_marker.png - Valid Marker Coordinates: [(379, 417), (549, 379), (276, 294)]
image2_marker.png - Knee Angle: 117.34°
image3_marker.png - Valid Marker Coordinates: [(377, 565), (348, 390), (482, 287)]
image3_marker.png - Knee Angle: 118.14°
image4_marker.png - Valid Marker Coordinates: [(380, 549), (342, 379), (465, 276)]
image4_marker.png - Knee Angle: 117.34°


#### MARKERLESS

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False, min_detection_confidence=0.5)

# Define the input and output directories
input_folder = r"C:\PJT2\Orbbec_Dataset_New\Orbbec_Dataset\validation\markerless"
output_folder = r"C:\PJT2\Orbbec_Dataset_New\Orbbec_Dataset\validation\markerless-output"

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get the list of image files in the dataset
image_files = [f for f in os.listdir(input_folder) if f.endswith('.png') or f.endswith('.jpg')]

# Function to calculate angle between three points (hip, knee, ankle)
def calculate_angle(a, b, c):
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
    return angle

# Loop through each image in the dataset
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Image '{image_file}' not found or failed to load.")
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image using MediaPipe Pose
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        left_hip = (int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * image.shape[1]),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * image.shape[0]))
        left_knee = (int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * image.shape[1]),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * image.shape[0]))
        left_ankle = (int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x * image.shape[1]),
                      int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y * image.shape[0]))

        knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

        # **Step 1: Draw lines first**
        cv2.line(image, left_hip, left_knee, (0, 0, 0), 2)
        cv2.line(image, left_knee, left_ankle, (0, 0, 0), 2)

        # **Step 2: Draw keypoints next**
        keypoints = [left_hip, left_knee, left_ankle]
        for point in keypoints:
            cv2.circle(image, point, 8, (255, 0, 0), -1)  # Blue circle

        # **Step 3: Draw coordinate points after the lines**
        for point in keypoints:
            text = f"{point}"
            text_x, text_y = point[0] + 10, point[1] - 10

            # Black outline
            cv2.putText(image, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 3, cv2.LINE_AA)
            # White text
            cv2.putText(image, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        # **Step 4: Draw knee angle arc in black**
        center = left_knee
        radius = 40
        start_angle = int(np.degrees(np.arctan2(left_hip[1] - left_knee[1], left_hip[0] - left_knee[0])))
        end_angle = start_angle + int(knee_angle)
        cv2.ellipse(image, center, (radius, radius), 0, start_angle, end_angle, (0, 0, 0), 2)  # Black arc

        # **Step 5: Display knee angle in deep red**
        text_position = (left_knee[0] + 50, left_knee[1] + 20)
        cv2.putText(image, f"{knee_angle:.2f} deg", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 139), 2, cv2.LINE_AA)  # Deep red text

        # Print detected keypoints and knee angle
        print(f"\nProcessing Image: {image_file}")
        print("Detected Keypoints:")
        for coord in keypoints:
            print(f"{coord}")
        print(f"\nCalculated Knee Angle: {knee_angle:.2f}°")

        # Save and display the output image
        output_image_path = os.path.join(output_folder, f"processed_{image_file}")
        cv2.imwrite(output_image_path, image)

    else:
        print(f"No keypoints detected in {image_file}.")

# End of code



Processing Image: image1_markerless.png
Detected Keypoints:
(293, 271)
(395, 412)
(565, 388)

Calculated Knee Angle: 117.85°

Processing Image: image2_markerless.png
Detected Keypoints:
(283, 285)
(388, 420)
(545, 379)

Calculated Knee Angle: 113.24°

Processing Image: image3_markerless.png
Detected Keypoints:
(490, 290)
(346, 390)
(372, 561)

Calculated Knee Angle: 116.13°

Processing Image: image4_markerless.png
Detected Keypoints:
(469, 287)
(338, 389)
(379, 541)

Calculated Knee Angle: 112.81°
